# Merging, Joining, Concatenating DataFrames

In [1]:
import pandas as pd

In [4]:
# Read in the datasets we will be using
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv")
food = pd.read_csv("Restaurant - Foods.csv")

### In this dataset, the keys are the following: 
#### Customer ID, Food ID 

In [10]:
week1.head(1)

,Customer ID,Food ID
0,537,9


In [11]:
food.head(1)

,Food ID,Food Item,Price
0,1,Sushi,3.99


In [13]:
# Note that in CUSTOMERS table, Customer ID is just ID
customers.head(1)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist


## Using `.concat()` Method
#### Concat will preserve the original DataFrame Indices by default

In [23]:
# Concat 2 dataframes
# Note that the indices are preserved
pd.concat(objs = [week1, week2])

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [24]:
# Use the ignore_index parameter to replace index w/ new numbered index
pd.concat(objs = [week1, week2], ignore_index=True)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [26]:
# .append() Method is deprecated, but functions the same as .concat()
week1.append(week2).head(3)

C:\Users\Smith\AppData\Local\Temp\ipykernel_14116\3779865938.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  week1.append(week2).head(3)


,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


# The `.concat()` Method pt 2 - MultiIndex
### What if we want to preserve the original index labels, but also add a unified index?
#### For example, we want to remember what came from week1 and week2

In [30]:
sales = pd.concat(objs=[week1, week2], keys=["week1", "week2"])
sales

Customer ID  Food ID
week1 0            537        9
      1             97        4
      2            658        1
      3            202        2
      4            155        9
...                ...      ...
week2 245          783       10
      246          556       10
      247          547        9
      248          252        9
      249          249        6

[500 rows x 2 columns]

In [32]:
# Grab the data from week 1 only
sales.loc[("week1",)]

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


In [33]:
# Grab the data from week 2 and index position 240
sales.loc[("week2", 240)]

Customer ID    734
Food ID          1
Name: (week2, 240), dtype: int64

In [34]:
# Isolate week 2, index 240, customer ID
sales.loc[("week2", 240), "Customer ID"]

734

# Inner Joins, Part 1
### Mimics SQL table behavior

In [41]:
# Week 1 is the left DF, Week 2 is the right DF
week1.merge(right=week2, how="inner", on="Customer ID").head(5)

,Customer ID,Food ID_x,Food ID_y
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9


In order to avoid duplicate column names, Pandas will provide an appended char (x, y)

In [39]:
# So this customer came in twice on week 1
week1[week1["Customer ID"] == 155]

,Customer ID,Food ID
4,155,9
17,155,1


In [42]:
# And once on week 2
week2[week2["Customer ID"] == 155]

,Customer ID,Food ID
208,155,3


Pandas doesn't know whether to append the Food ID 3 from week2 on the first or second value, so it does both. This can result in unexpected behavior, as it will look the same if that customer came in and ordered Food ID 3 twice on Week 2. AKA, we have introduced a duplicate into our data. So be wary I guess, he hasn't told us how to fix that.

In [45]:
# Changing the suffix names
week1.merge(right=week2, how="inner", on="Customer ID", suffixes=[" - Week 1", " - Week 2"], ).head(5)

,Customer ID,Food ID - Week 1,Food ID - Week 2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9


# Inner Joins, Part 2

In [49]:
# Merge where customer ID + food ID are the same between week1 and week2
week1.merge(week2, how="inner", on=["Customer ID", "Food ID"])

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


#### Once again, this can be misleading...

In [50]:
week1[week1["Customer ID"] == 578]

,Customer ID,Food ID
224,578,5


In [51]:
week2[week2["Customer ID"] == 578]

,Customer ID,Food ID
29,578,5
189,578,5


In [52]:
week1[week1["Customer ID"] == 21]

,Customer ID,Food ID
101,21,4
212,21,4


In [54]:
week2[week2["Customer ID"] == 21]

,Customer ID,Food ID
30,21,4


#### Notice that 578 bought the same food twice on week 2, while 21 bought the same food twice on week 1. However, in the merged dataframe, they look exactly the same. So, basically, this is not a good method for counting things. Probably better for boolean operations.

# Outer Joins

In [60]:
week1.merge(week2, how="outer", on="Customer ID", suffixes=[" - Week 1", " - Week 2"])

,Customer ID,Food ID - Week 1,Food ID - Week 2
0,537,9.0,5.0
1,97,4.0,NaN
2,658,1.0,NaN
3,202,2.0,NaN
4,155,9.0,3.0
...,...,...,...
449,855,NaN,4.0
450,559,NaN,10.0
451,276,NaN,4.0
452,556,NaN,10.0


In [62]:
merged = week1.merge(week2, how="outer", on="Customer ID", suffixes=[" - Week 1", " - Week 2"], indicator=True)
merged

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


In [63]:
merged["_merge"].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

In [65]:
mask = merged["_merge"].isin(["left_only", "right_only"])
merged[mask]

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


# Left Joins
### Closest thing to a vlookup you can get

In [67]:
food.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [68]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [71]:
# Merge food with week1 data, using Food ID as the foreign key
week1.merge(food, how="left", on="Food ID")

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99
...,...,...,...,...
245,413,9,Donut,0.99
246,926,6,Pasta,13.99
247,134,3,Taco,2.99
248,396,6,Pasta,13.99


In [75]:
# If you use the sort param, it will sort on the specified key
week1.merge(food, how="left", on="Food ID", sort=True)

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
...,...,...,...,...
245,809,10,Drink,1.75
246,584,10,Drink,1.75
247,274,10,Drink,1.75
248,151,10,Drink,1.75


# Merging when the column names are different between dataframes
### Choosing which columns to key on when the names aren't the same

In [78]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [77]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


### Notice that in the customers DF, our key is just called "ID", while in week2 its called "Customer ID"
We will use the left_on and right_on params to tell Pandas where to key within each DF

In [82]:
week2 = week2.merge(customers, how="left", left_on="Customer ID", right_on="ID").drop("ID", axis="columns")
week2

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,Roger,Gordon,Male,Skilith,Operator
4,267,3,Matthew,Wood,Male,Agimba,Product Engineer
...,...,...,...,...,...,...,...
245,783,10,Phyllis,Meyer,Female,Voolia,Information Systems Manager
246,556,10,Samuel,Bailey,Male,Oyoloo,Nurse
247,547,9,Tina,Watkins,Female,Thoughtstorm,Accountant II
248,252,9,Douglas,Powell,Male,Jetwire,Geologist IV


In [85]:
# Let's add in the food details too!
week2.merge(food, how="left", on="Food ID").sort_values("Customer ID")

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
16,8,6,Frances,Adams,Female,Dabshots,Developer III,Pasta,13.99
24,13,2,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer,Burrito,9.99
30,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive,Quesadilla,4.25
128,24,8,Donna,Thomas,Female,Jaxbean,Chief Design Engineer,Salad,11.25
197,27,4,Jessica,Bennett,Female,Twitternation,Account Executive,Quesadilla,4.25
...,...,...,...,...,...,...,...,...,...
150,968,4,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV,Quesadilla,4.25
55,969,3,Marie,Ortiz,Female,Meejo,Research Associate,Taco,2.99
168,977,7,Cynthia,Dixon,Female,Skalith,Automation Specialist II,Steak,24.99
108,994,2,Clarence,Morgan,Male,Edgewire,Geologist II,Burrito,9.99


# Merging on Indices with the `left_index` and `right_index` parameters

In [93]:
# Reimport data so none of the above changes mess me up
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv", index_col="ID") # Setting ID as the index
food = pd.read_csv("Restaurant - Foods.csv", index_col="Food ID") # Setting Food ID as the index

In [94]:
food.head(3)

,Food Item,Price
Food ID,,
1,Sushi,3.99
2,Burrito,9.99
3,Taco,2.99


In [100]:
# By using right_index, we tell Pandas to use the index as the key
# No duplicate columns are left over after merging this way either
sales = week1.merge(customers, how="left", left_on="Customer ID", right_index=True)
sales

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer


In [103]:
# Now merge in Food data the same way
sales = sales.merge(food, how="left", left_on="Food ID", right_index=True)
sales.head(3)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item_x,Price_x,Food Item_y,Price_y
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99,Donut,0.99
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25,Quesadilla,4.25
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99,Sushi,3.99


In [104]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [105]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [107]:
# Now lets merge on a common index, not any columns
week1.merge(week2, how="left", left_index=True, right_index=True, suffixes=[" - Week 1", " - Week 2"])

,Customer ID - Week 1,Food ID - Week 1,Customer ID - Week 2,Food ID - Week 2
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
...,...,...,...,...
245,413,9,783,10
246,926,6,556,10
247,134,3,547,9
248,396,6,252,9


# The `.join()` Method
### Shortcut for merging on index

In [108]:
# Reimport data so none of the above changes mess me up
# Also import another file called satisfaction
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv")
food = pd.read_csv("Restaurant - Foods.csv")
satisfaction = pd.read_csv("Restaurant - Week 1 Satisfaction.csv")

In [109]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [110]:
satisfaction.head(3)

,Satisfaction Rating
0,2
1,7
2,3


In [114]:
# The syntax heavy way of doing it, as seen above
week1.merge(satisfaction, how="left", left_index=True, right_index=True).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


In [116]:
# Here is the easier way
week1.join(satisfaction).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


# Another syntax for merging, `pd.merge()`

In [117]:
pd.merge(left=week1, right=week2, how="inner", on="Customer ID", suffixes=[" - Week 1", " - Week 2"])

,Customer ID,Food ID - Week 1,Food ID - Week 2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7
